# ThD Cases

In [1]:
import os, sys, shutil, math
import numpy as np
from shutil import rmtree, copy
from matplotlib import pyplot as plt
from matplotlib import gridspec, cm
from PIL import Image, ImageDraw, ImageFont
from scipy.interpolate import interp1d
from scipy.interpolate import UnivariateSpline
import datetime

# Include this pakage
HaMaGeoLib_DIR = "/home/lochy/ASPECT_PROJECT/HaMaGeoLib"
if os.path.abspath(HaMaGeoLib_DIR) not in sys.path:
    sys.path.append(os.path.abspath(HaMaGeoLib_DIR))
from hamageolib.utils.exception_handler import my_assert

# Working directories
local_TwoDSubduction_dir = "/mnt/lochz/ASPECT_DATA/TwoDSubduction"
local_ThDSubduction_dir = "/mnt/lochy/ASPECT_DATA/ThDSubduction" # data directory
# local_ThDSubduction_dir = "/mnt/lochy2/ASPECT_DATA/ThDSubduction" # data back directory
remote_ThDSubduction_dir = "peloton:/group/billengrp-mpi-io/lochy/ThDSubduction"
assert(os.path.isdir(local_ThDSubduction_dir))

# check case name and local directory
case_name_2d = None # set an None initial value to determine whether the 3d case is connected to a 2d case
local_dir_2d = None

# py_temp file and temperature results directory
py_temp_dir = os.path.join(HaMaGeoLib_DIR, "py_temp_files")
RESULT_DIR = os.path.join(HaMaGeoLib_DIR, 'results')
os.makedirs(py_temp_dir, exist_ok=True) # Ensure the directory exists

today_date = datetime.datetime.today().strftime("%Y-%m-%d") # Get today's date in YYYY-MM-DD format
py_temp_file = os.path.join(py_temp_dir, f"py_temp_{today_date}.sh")

if not os.path.exists(py_temp_file):
    bash_header = """#!/bin/bash
# =====================================================
# Script: py_temp.sh
# Generated on: {date}
# Description: Temporary Bash script created by Python
# =====================================================

""".format(date=today_date)
    with open(py_temp_file, "w") as f:
        f.write(bash_header)

print(f"File ensured at: {py_temp_file}")

File ensured at: /home/lochy/ASPECT_PROJECT/HaMaGeoLib/py_temp_files/py_temp_2025-07-17.sh


# Utility functions

In [2]:
def get_slab_dimensions_3(x, y, z, Ro, is_chunk):
    '''
    Derives the length along the three dimensions of a subducting slab.

    Inputs:
        x (float): x-coordinate of the slab point.
        y (float): y-coordinate of the slab point.
        z (float): z-coordinate of the slab point.
        Ro (float): Outer radius of the spherical domain.
        is_chunk (bool): Flag indicating whether the geometry is a spherical chunk.

    Returns:
        tuple: A tuple containing (r, w, l):
            - r (float): Radius or z-coordinate depending on whether the geometry is a chunk.
            - w (float): Width of the slab in the y-dimension, or converted width for chunk geometry.
            - l (float): Length of the slab in the x-dimension, or converted length for chunk geometry.
    
    Description:
        - For chunk geometries, converts Cartesian coordinates to spherical coordinates and calculates
          width and length using the outer radius Ro and spherical angles.
        - For non-chunk geometries, returns the z, x, and y coordinates directly as radius, length, and width.
    '''
    if is_chunk:
        # Convert Cartesian coordinates to spherical coordinates for chunk geometry
        r, th1, ph1 = Utilities.cart2sph(x, y, z)
        w = Ro * (np.pi / 2.0 - th1)  # Calculate width using the spherical angle th1
        l = Ro * ph1  # Calculate length using the spherical angle ph1
    else:
        # For non-chunk geometry, use Cartesian coordinates directly
        r = z
        l = x
        w = y
    
    return r, w, l


def GetSlabDipAngle(case_dir, time_interval_for_slab_morphology, **kwargs):
    '''
    Plots trench position for a single time step and calculates the slab dip angle.

    Inputs:
        case_dir (str): Directory containing the case data.
        time_interval_for_slab_morphology (float): Time interval used to prepare snapshots for slab morphology analysis.
        kwargs (dict): Additional options:
            - silence (bool, default=False): Suppresses warnings if set to True.
            - w_query (float, default=0.0): Width value used for querying trench position.
            - pin_depth (float, default=100e3): Depth value for pinning.
            - crust_only (int, default=0): Flag to specify crustal composition to consider (0: both, 1: upper, 2: lower).

    Returns:
        tuple: Three numpy arrays containing time steps, slab depths, and dip angles.
    
    Description:
        - Initializes VTK options and extracts trench and slab tip positions over multiple time snapshots.
        - Interpolates data to derive the slab dip angle at each time snapshot.
        - Handles missing data files with warnings (unless silenced) and skips to the next snapshot.
    '''
    silence = kwargs.get("silence", False)
    w_query = kwargs.get("w_query", 0.0)
    pin_depth = kwargs.get("pin_depth", 100e3)
    crust_only = kwargs.get("crust_only", 0)

    # Initialize VTK options and interpret case settings
    Visit_Options = VISIT_OPTIONS(case_dir)
    Visit_Options.Interpret()
    trench_initial_position = Visit_Options.options['TRENCH_INITIAL']

    # Prepare snapshots for slab morphology with a given time interval
    available_pvtu_snapshots = Visit_Options.get_snaps_for_slab_morphology(time_interval=time_interval_for_slab_morphology)
    n_snapshots = len(available_pvtu_snapshots)

    is_chunk = (Visit_Options.options['GEOMETRY'] == "chunk")
    Ro = Visit_Options.options['OUTER_RADIUS']

    # Lists to store results
    dips = []
    depths = []
    ts = []

    # Iterate through each snapshot to calculate dip angles
    for n in range(n_snapshots - 1):
        vtu_snapshot = available_pvtu_snapshots[n]

        # Construct file names based on crust_only flag
        if crust_only == 1:
            filename0 = "trench_b_%05d.txt" % vtu_snapshot
            filename1 = "center_profile_%05d.txt" % vtu_snapshot
            filename2 = "trench_b_d%.2fkm_%05d.txt" % (pin_depth / 1e3, vtu_snapshot)
        elif crust_only == 2:
            filename0 = "trench_l_b_%05d.txt" % vtu_snapshot
            filename1 = "center_profile_l_%05d.txt" % vtu_snapshot
            filename2 = "trench_l_b_d%.2fkm_%05d.txt" % (pin_depth / 1e3, vtu_snapshot)
        else:
            filename0 = "trench_lu_b_%05d.txt" % vtu_snapshot
            filename1 = "center_profile_lu_%05d.txt" % vtu_snapshot
            filename2 = "trench_lu_b_d%.2fkm_%05d.txt" % (pin_depth / 1e3, vtu_snapshot)

        # Construct file paths
        filein0 = os.path.join(case_dir, "vtk_outputs", filename0)
        filein1 = os.path.join(case_dir, "vtk_outputs", filename1)
        filein2 = os.path.join(case_dir, "vtk_outputs", filename2)

        # Check if files exist, issue warnings if not found
        if not os.path.isfile(filein0):
            if not silence:
                warnings.warn(f'PlotTrenchVelocity: File {filein0} is not found', UserWarning)
            continue
        if not os.path.isfile(filein1):
            if not silence:
                warnings.warn(f'PlotTrenchVelocity: File {filein1} is not found', UserWarning)
            continue

        # Get time and step information for the current snapshot
        _time, step = Visit_Options.get_time_and_step_by_snapshot(vtu_snapshot)

        # Load trench position data from file
        data0 = np.loadtxt(filein0)
        xs0, ys0, zs0 = data0[:, 0], data0[:, 1], data0[:, 2]
        rs0, ws0, ls0 = get_slab_dimensions_3(xs0, ys0, zs0, Ro, is_chunk)
        l_tr0 = np.interp(w_query, ws0, ls0)
        r_tr0 = np.interp(w_query, ws0, rs0)

        # Load slab tip depth data from file
        data1 = np.loadtxt(filein1)
        rs1 = data1[:, 2]
        depth = float(Visit_Options.options["OUTER_RADIUS"]) - np.min(rs1)
        depths.append(depth)
        ts.append(_time)

        # Load data for the curve at a specific depth
        data2 = np.loadtxt(filein2)
        xs2, ys2, zs2 = data2[:, 0], data2[:, 1], data2[:, 2]
        rs2, ws2, ls2 = get_slab_dimensions_3(xs2, ys2, zs2, Ro, is_chunk)
        l_tr2 = np.interp(w_query, ws2, ls2)
        r_tr2 = np.interp(w_query, ws2, rs2)

        # Calculate the dip angle and append to the list
        dip = -math.atan((r_tr0 - r_tr2) / (l_tr0 - l_tr2))
        print(f"vtu_snapshot = {vtu_snapshot}, l_tr0 = {l_tr0:.4e}, r_tr0 = {r_tr0:.4e}, l_tr2 = {l_tr2:.4e}, r_tr2 = {r_tr2:.4e}, dip = {dip:.4e}")  # Debug output
        dips.append(dip)

    # Convert lists to numpy arrays
    dips = np.array(dips)
    depths = np.array(depths)
    ts = np.array(ts)

    return ts, depths, dips


def PlotSlabDipAngle(case_dir, time_interval_for_slab_morphology, **kwargs):
    '''
    Plots the slab dip angle and trench position for a single time step.

    Inputs:
        case_dir (str): Directory containing the case data.
        time_interval_for_slab_morphology (float): Time interval for preparing snapshots of slab morphology.
        kwargs (dict): Additional plotting options:
            - color (str or None): Color of the plot lines.
            - label (str or None): Label for the plot lines.
            - w_query (float, default=0.0): Query width value in y-dimension for plotting.
            - axis_twinx (matplotlib axis object or None): Axis object for plotting depth on a secondary y-axis.
            - axis (matplotlib axis object, required): Axis object for plotting the dip angle.
    
    Returns:
        matplotlib axis object: The axis used for plotting the dip angle.

    Description:
        - Retrieves time, depths, and dip angles using `GetSlabDipAngle`.
        - Uses a univariate spline to smooth the dip angle data for visualization.
        - Plots the original and splined dip angles over time.
        - If `w_query` is near zero, also plots the depth at the center on the secondary y-axis.
    '''

    _color = kwargs.get("color", None)
    _label = kwargs.get("label", None)
    w_query = kwargs.get("w_query", 0.0)
    ax_twinx = kwargs.get("axis_twinx", None)

    # Initiate the plot on the provided axis
    ax = kwargs.get('axis', None)
    if ax is None:
        raise ValueError("Axis object must be provided for plotting.")

    # Retrieve time, depths, and dip angles
    ts, depths, dips = GetSlabDipAngle(case_dir, time_interval_for_slab_morphology, **kwargs)
    # Plot the dip angles, converting from radians to degrees
    ax.plot(ts / 1e6, dips * 180.0 / np.pi, label="y = %.2f km" % (w_query / 1e3), color=_color)

    # Apply a univariate spline to smooth the dip angles
    # size of data must be bigger than 3 to make a spline
    assert(ts.size > 3)
    spline = UnivariateSpline(ts / 1e6, dips, s=0)  # s=0 means interpolation without smoothing
    ts_new = np.linspace(ts.min(), ts.max(), 1000)
    dips_splined = spline(ts_new / 1e6)

    # Plot the splined dip angles, converting from radians to degrees
    ax.plot(ts_new / 1e6, dips_splined * 180.0 / np.pi, "-.", label="y = %.2f km (splined)" % (w_query / 1e3), color=_color)

    # If w_query is near zero, plot the depth at the center on the secondary y-axis
    if w_query < 1e-6 and ax_twinx is not None:
        ax_twinx.plot(ts / 1e6, depths / 1e3, "--", color=_color)  # Plot depth in km

    return ax



def PlotTrenchDifferences(case_dir, time_interval_for_slab_morphology, **kwargs):
    '''
    Plots the differences in trench position and the slab tip depth over time.

    Inputs:
        case_dir (str): Directory containing the case data.
        time_interval_for_slab_morphology (float): Time interval for preparing snapshots of slab morphology.
        kwargs (dict): Additional plotting options:
            - color (str or None): Color of the plot lines.
            - label (str or None): Label for the plot lines.
            - silence (bool, default=False): Suppresses warnings if set to True.
            - w_query (float, default=0.0): Width value used for querying trench position.
            - axis_twinx (matplotlib axis object or None): Axis object for plotting depth on a secondary y-axis.
            - axis (matplotlib axis object, required): Axis object for plotting trench differences.
    
    Returns:
        matplotlib axis object: The axis used for plotting the trench differences.

    Description:
        - Initializes VTK options and retrieves snapshots based on the specified time interval.
        - Loads trench position and slab tip depth data for each snapshot and interpolates the trench position.
        - Plots the difference in trench position from the initial value and, if applicable, plots the slab depth.
    '''
    _color = kwargs.get("color", None)
    _label = kwargs.get("label", None)
    silence = kwargs.get("silence", False)
    w_query = kwargs.get("w_query", 0.0)
    ax_twinx = kwargs.get("axis_twinx", None)

    # Initialize VTK options and interpret case settings
    Visit_Options = VISIT_OPTIONS(case_dir)
    Visit_Options.Interpret()
    trench_initial_position = Visit_Options.options['TRENCH_INITIAL']

    # Prepare snapshots for slab morphology with the given time interval
    available_pvtu_snapshots = Visit_Options.get_snaps_for_slab_morphology(time_interval=time_interval_for_slab_morphology)
    n_snapshots = len(available_pvtu_snapshots)

    is_chunk = (Visit_Options.options['GEOMETRY'] == "chunk")
    Ro = Visit_Options.options['OUTER_RADIUS']

    # Initialize the plot on the provided axis
    ax = kwargs.get('axis', None)
    if ax is None:
        raise ValueError("Axis object must be provided for plotting.")

    # Lists to store results
    dl_trs = []
    depths = []
    ts = []

    # Iterate through each snapshot to calculate trench differences
    for n in range(n_snapshots - 1):
        vtu_snapshot = available_pvtu_snapshots[n]

        # Construct file paths for trench position and slab tip depth
        filein0 = os.path.join(case_dir, "vtk_outputs", "trench_%05d.txt" % vtu_snapshot)
        filein1 = os.path.join(case_dir, "vtk_outputs", "center_profile_%05d.txt" % vtu_snapshot)

        # Check if files exist, issue warnings if not found
        if not os.path.isfile(filein0):
            if not silence:
                warnings.warn(f'PlotTrenchVelocity: File {filein0} is not found', UserWarning)
            continue
        if not os.path.isfile(filein1):
            if not silence:
                warnings.warn(f'PlotTrenchVelocity: File {filein1} is not found', UserWarning)
            continue

        # Get time and step information for the current snapshot
        _time, step = Visit_Options.get_time_and_step_by_snapshot(vtu_snapshot)

        # Load trench position data from file
        data0 = np.loadtxt(filein0)
        xs0, ys0, zs0 = data0[:, 0], data0[:, 1], data0[:, 2]
        rs0, ws0, ls0 = get_slab_dimensions_3(xs0, ys0, zs0, Ro, is_chunk)
        l_tr0 = np.interp(w_query, ws0, ls0)
        dl_trs.append(l_tr0)

        # Load slab tip depth data from file
        data1 = np.loadtxt(filein1)
        xs1, ys1, zs1 = data1[:, 0], data1[:, 1], data1[:, 2]
        rs1, ws1, ls1 = get_slab_dimensions_3(xs1, ys1, zs1, Ro, is_chunk)
        depth = Ro - np.min(rs1)
        depths.append(depth)
        ts.append(_time)

    # Convert lists to numpy arrays
    dl_trs = np.array(dl_trs)
    depths = np.array(depths)
    ts = np.array(ts)

    # Plot the differences in trench position, subtracting the initial value
    ax.plot(ts / 1e6, (dl_trs - dl_trs[0]) / 1e3, label="y = %.2f km" % (w_query / 1e3), color=_color)

    # If w_query is near zero, plot the depth at the center on the secondary y-axis
    if w_query < 1e-6 and ax_twinx is not None:
        ax_twinx.plot(ts / 1e6, depths / 1e3, "--", color=_color)  # Plot depth in km

    return ax, ax_twinx


# Case name

case_name is the relative path to local_TwoDSubduction_dir, and local_ThDSubduction_dir

In [3]:
# case_name = "EBA_2d_consistent_7/eba3d_width61_c22_AR4"
# case_name = "EBA_2d_consistent_8/eba3d_width61_c22_AR4"
# case_name = "EBA_2d_consistent_8_1/eba3d_width61_c22_AR4"
# case_name = "EBA_2d_consistent_8_2/eba3d_width61_c22_AR4"
# case_name = "EBA_2d_consistent_8_3/eba3d_width61_c22_AR4"
# case_name = "EBA_2d_consistent_8_2/eba3d_width61_c23_AR4"
# case_name = "EBA_2d_consistent_8_2/eba3d_width61_c22_AR4_cd150"
# case_name = "EBA_2d_consistent_8_4/eba3d_width61_c22_AR4"
# case_name = "EBA_2d_consistent_8_4/eba3d_width61_c22_AR4_old_rheology"
# case_name = "EBA_2d_consistent_8_4/eba3d_width61_c22_AR4_old_rheology_old_density"
# case_name = "EBA_2d_consistent_8_4/eba3d_width61_c22_AR4_old_density"
# case_name = "EBA_2d_consistent_8_5/eba3d_width61_c22_AR4_fix_density"

# Cases without a Peierls creep
# case_name = None; case_name_2d = "EBA_CDPT_3dconsistent_9/eba_cdpt_coh300_SA80.0_OA40.0_width80_sc22"

# Cases with the Peierls creep, a deep box
# case_name = "EBA_2d_consistent_8_6/eba3d_width51_c22_AR4"; case_name_2d = "EBA_CDPT_3dconsistent_9/eba_cdpt_coh300_SA80.0_OA40.0_width51_sc22"
# case_name = "EBA_2d_consistent_8_6/eba3d_width61_c22_AR4"; case_name_2d = "EBA_CDPT_3dconsistent_9/eba_cdpt_coh300_SA80.0_OA40.0_width61_sc22"
# case_name = "EBA_2d_consistent_8_6/eba3d_width61_c22_AR3"
# case_name = "EBA_2d_consistent_8_6/eba3d_width61_c23_AR4"
# case_name = "EBA_2d_consistent_8_6/eba3d_width80_c22_AR4"; case_name_2d = "EBA_CDPT_3dconsistent_9/eba_cdpt_coh300_SA80.0_OA40.0_width80_sc22"
# case_name ="EBA_2d_consistent_8_6/eba3d_width61_c22_AR5"
# case_name = "EBA_2d_consistent_8_8/eba3d_width80_bw8000_sw2000_c22_AR4_yd300.0"; case_name_2d = "EBA_CDPT_3dconsistent_13/eba_cdpt_coh300_SA80.0_OA40.0_width80_ss300.0"
# case_name = None; case_name_2d = "EBA_CDPT_3dconsistent_9/eba_cdpt_coh300_SA80.0_OA40.0_width80_sc22_nlht"
# case_name = None; case_name_2d = "EBA_CDPT_3dconsistent_10/eba_cdpt_coh300_SA80.0_OA40.0_width80_sc22_ss100.0"
# case_name = "EBA_2d_consistent_8_6/eba3d_width140_c22_AR4"
# case_name = "EBA_2d_consistent_9/eba3d_width80_h2000"
# case_name = "EBA_2d_consistent_8_7/eba3d_width80_bw2000_sw500_c22_AR4" ; 
# case_name = "EBA_2d_consistent_8_7/eba3d_width80_bw8000_sw2000_c22_AR4"; case_name_2d = "EBA_CDPT_3dconsistent_9/eba_cdpt_coh300_SA80.0_OA40.0_width80_sc22"
# case_name = "EBA_2d_consistent_8_8/eba3d_width51_bw4000_sw1000_c22_AR4_yd300.0"; case_name_2d = "EBA_CDPT_3dconsistent_10/eba_cdpt_coh300_SA80.0_OA40.0_width51_sc22_ss300.0"
# case_name = "EBA_2d_consistent_8_8/eba3d_width80_bw2000_sw500_c22_AR4_yd300.0"; 

# Cases with the Peierls creep, a shallow box
# case_name = None; case_name_2d = "EBA_CDPT_3dconsistent_12/eba_cdpt_coh300_SA40.0_OA20.0_width80_h1000_ss300.0"
# case_name = None; case_name_2d = "EBA_CDPT_3dconsistent_12/eba_cdpt_coh300_SA80.0_OA40.0_width80_h1000_ss300.0"
# case_name = "EBA_2d_consistent_8_6/eba3d_width80_c22_AR4_yd100"; case_name_2d = "EBA_CDPT_3dconsistent_13/eba_cdpt_coh300_SA80.0_OA40.0_width80_ss100.0"
# case_name = "EBA_2d_consistent_8_6/eba3d_width80_c22_AR4_yd300"; case_name_2d = "EBA_CDPT_3dconsistent_13/eba_cdpt_coh300_SA80.0_OA40.0_width80_ss300.0"
# case_name = "EBA_2d_consistent_9/eba3d_width80_h1000"; case_name_2d = "EBA_CDPT_3dconsistent_13/eba_cdpt_coh300_SA80.0_OA40.0_width80_h1000_ss1000000.0"
# case_name = "EBA_2d_consistent_9/eba3d_width80_h2000"; case_name_2d = None

# case_name = "chunk_test/chunk_initial0"
# case_name = "chunk_test/chunk_initial1"
# case_name = "chunk_test/chunk_initial2"
# case_name = "chunk_test/chunk_initial3"
# case_name = "chunk_test/chunk_initial4"
# case_name = "chunk_test/chunk_initial5"
# case_name = "chunk_test/chunk_initial6"
# case_name = "chunk_test/chunk_initial7"
# case_name = "chunk_test/chunk_initial8"
# case_name = "chunk_test/chunk_initial8_T"
# case_name = "chunk_test/chunk_initial9"; case_name_2d = "EBA_CDPT_3dconsistent_chunk_1/eba_cdpt_coh300_SA80.0_OA40.0_width61_ss1000000.0"
# case_name = "chunk_test/chunk_initial9_re_test_1"; case_name_2d = "EBA_CDPT_3dconsistent_chunk_1/eba_cdpt_coh300_SA80.0_OA40.0_width61_ss1000000.0"
# case_name = None; case_name_2d = "EBA_CDPT_3dconsistent_chunk_1/eba_cdpt_coh300_SA80.0_OA40.0_width51_ss1000000.0"
# case_name = None; case_name_2d = "EBA_CDPT_3dconsistent_chunk_1/eba_cdpt_coh300_SA80.0_OA40.0_width80_ss1000000.0"
# case_name = None; case_name_2d = "EBA_CDPT_3dconsistent_chunk_1/eba_cdpt_coh300_SA80.0_OA40.0_width140_ss1000000.0"
# case_name = None; case_name_2d = "EBA_CDPT_3dconsistent_chunk_1/eba_cdpt_coh300_SA40.0_OA20.0_width80_ss1000000.0"
# case_name = "chunk_test/chunk_initial9_re_test"
case_name = "chunk_geometry1/eba3d_width80_bw4000_sw1000_yd500.0_AR4"; case_name_2d = "EBA_CDPT_3dconsistent_chunk_1/eba_cdpt_coh300_SA80.0_OA40.0_width80_ss500.0"
# case_name = "chunk_geometry1/eba3d_width80_bw4000_sw1000_yd100.0_AR4"; case_name_2d = "EBA_CDPT_3dconsistent_chunk_1/eba_cdpt_coh300_SA80.0_OA40.0_width80_ss100.0"
# case_name = "chunk_geometry1/eba3d_width80_bw8000_sw2000_yd500.0_AR4"; case_name_2d = "EBA_CDPT_3dconsistent_chunk_1/eba_cdpt_coh300_SA80.0_OA40.0_width80_ss500.0"

local_dir = None; local_dir_2d = None
if case_name is not None:
    local_dir = os.path.join(local_ThDSubduction_dir, case_name)
    assert(os.path.isdir(local_dir))
    print("local_dir:\n\t", local_dir)
if case_name_2d is not None:
    local_dir_2d = os.path.join(local_TwoDSubduction_dir, case_name_2d)
    assert(os.path.isdir(local_dir_2d))
    print("local_dir_2d:\n\t", local_dir_2d)

local_dir:
	 /mnt/lochy/ASPECT_DATA/ThDSubduction/chunk_geometry1/eba3d_width80_bw4000_sw1000_yd500.0_AR4
local_dir_2d:
	 /mnt/lochz/ASPECT_DATA/TwoDSubduction/EBA_CDPT_3dconsistent_chunk_1/eba_cdpt_coh300_SA80.0_OA40.0_width80_ss500.0


# Visualization

The next block will generate plots of runtime, solver statistics, etc., and will compose a script to run in ParaView.

## 3d case

- Processing vtu data with pyvista
- Generate paraview scritps

### Generate Paraview script

In [4]:
is_prepare_for_plot = False
is_process_pyvista_for_plot = False

if is_prepare_for_plot:

    from hamageolib.research.haoyuan_3d_subduction.post_process import get_trench_position_from_file, get_slab_depth_from_file, PLOT_CASE_RUN_THD
    from hamageolib.research.haoyuan_3d_subduction.case_options import CASE_OPTIONS
    
    assert(local_dir is not None)

    # parameters
    ofile_list = ["slab1.py"]; require_base=True
    time_range = None
    time_interval = None
    # turn on plot_axis if I want to save a complete result
    # turn off if I want to prepare for figures in a paper
    plot_axis = False
    graphical_steps = [161]; slices=None # specify steps
    # step = "auto"; slices=3  # auto-figure out the steps, take the numebr of slices
    max_velocity = -1.0  # rescale the color for velocity
    rotation_plus = 0.47 # rotation of the frame along the lon when making plot

    # case options 
    Case_Options = CASE_OPTIONS(local_dir)
    Case_Options.Interpret()
    Case_Options.SummaryCaseVtuStep(os.path.join(local_dir, "summary.csv"))

    # Initiate plotting class
    PlotCaseRunThD = PLOT_CASE_RUN_THD(local_dir, time_range=time_range, run_visual=False,\
            time_interval=time_interval, visualization="paraview", step=graphical_steps, plot_axis=plot_axis, max_velocity=max_velocity,\
                    rotation_plus=rotation_plus, ofile_list=ofile_list, require_base=require_base)

    # Processing pyvista
    if is_process_pyvista_for_plot:
        PlotCaseRunThD.ProcessPyvista()

    # Generate paraview script
    for step in graphical_steps:
        my_assert(len(graphical_steps)==1, ValueError, "Feeding the trench position only works when there is only one step")

        # Get time 
        idx = Case_Options.summary_df["Vtu step"] == step
        _time = Case_Options.summary_df.loc[idx, "Time"].values[0]

        # get trench center
        pvtu_step = step + int(Case_Options.options['INITIAL_ADAPTIVE_REFINEMENT']) 
        pyvista_outdir = os.path.join(local_dir, "pyvista_outputs", "%05d" % pvtu_step)
        trench_center = get_trench_position_from_file(pyvista_outdir, pvtu_step, Case_Options.options['GEOMETRY'])
        slab_depth = get_slab_depth_from_file(pyvista_outdir, pvtu_step, Case_Options.options['GEOMETRY'], float(Case_Options.options['OUTER_RADIUS']), "sp_lower")

        print("trench_center: ", trench_center) # debug
        # generate paraview script
        addtional_options = {"TRENCH_CENTER": trench_center, "PLOT_TIME": _time}
        PlotCaseRunThD.GenerateParaviewScript(ofile_list, addtional_options)

### Automazed workflow to finalize visualization

In [5]:
finalize_visual = False

if finalize_visual:

    from IPython.display import Image, display
    from hamageolib.research.haoyuan_2d_subduction.workflow_scripts import finalize_visualization_2d_12172024

    _time = 16.1e6
    
    # file types
    file_name = "slice_center_viscosity"

    if file_name in ["slice_center_viscosity", "T", "density", "metastable"]:
        frame_png_file_with_ticks = "/home/lochy/Documents/papers/documented_files/ThDSubduction/Frame/upper_mantle_frame_12172024_trans_modified-01.png"
    
        output_image_file = finalize_visualization_2d_12172024(local_dir, file_name, _time, frame_png_file_with_ticks, add_time=False)

### Analyze Slab Morphology

A separate script is generated to analyze slab morphology for all time steps.
Here the code block serves to analyze a single block.

In [6]:
is_slab_morphology = False
is_process_pyvista_for_slab_morphology = False

if is_slab_morphology:

    # Initiate the case option class
    from hamageolib.research.haoyuan_3d_subduction.post_process import get_trench_position_from_file, get_slab_depth_from_file, PLOT_CASE_RUN_THD
    from hamageolib.research.haoyuan_3d_subduction.case_options import CASE_OPTIONS

    Case_Options = CASE_OPTIONS(local_dir)
    Case_Options.Interpret()
    Case_Options.SummaryCaseVtuStep(os.path.join(local_dir, "summary.csv"))

    graphical_steps_np = Case_Options.summary_df["Vtu step"].to_numpy()
    graphical_steps = [int(step) for step in graphical_steps_np]

    # Initiate plotting class
    PlotCaseRunThD = PLOT_CASE_RUN_THD(local_dir, time_range=time_range, run_visual=False,\
            time_interval=time_interval, visualization="paraview", step=graphical_steps, plot_axis=plot_axis, max_velocity=max_velocity,\
                    rotation_plus=rotation_plus, ofile_list=ofile_list, require_base=require_base)

    # Processing pyvista
    if is_process_pyvista_for_slab_morphology:
        PlotCaseRunThD.ProcessPyvista()

    # Generate paraview script
    for step in graphical_steps:
        # get trench center
        pvtu_step = step + int(Case_Options.options['INITIAL_ADAPTIVE_REFINEMENT']) 
        pyvista_outdir = os.path.join(local_dir, "pyvista_outputs", "%05d" % pvtu_step)
        trench_center = get_trench_position_from_file(pyvista_outdir, pvtu_step, Case_Options.options['GEOMETRY'])
        slab_depth = get_slab_depth_from_file(pyvista_outdir, pvtu_step, Case_Options.options['GEOMETRY'], float(Case_Options.options['OUTER_RADIUS']), "sp_lower")
        # generate paraview script
        addtional_options = {"TRENCH_CENTER": trench_center}
        PlotCaseRunThD.GenerateParaviewScript(ofile_list, addtional_options)
        # update value in sumamry
        Case_Options.SummaryCaseVtuStepUpdateValue("Slab depth", step, slab_depth)
        Case_Options.SummaryCaseVtuStepUpdateValue("Trench (center)", step, trench_center)

        break # debug

    Case_Options.SummaryCaseVtuStepExport(os.path.join(local_dir, "summary.csv"))

## 2d case

In [ ]:
# todo_2d_visual
is_prepare_for_plot_2d = True
is_process_pyvista_for_plot_2d = False

if is_prepare_for_plot_2d:
    from hamageolib.research.haoyuan_3d_subduction.case_options import CASE_OPTIONS_TWOD1
    from hamageolib.research.haoyuan_3d_subduction.post_process import PlotCaseRunTwoD1
    from hamageolib.research.haoyuan_2d_subduction.workflow_scripts import run_2d_subduction_visualization

    assert(local_dir_2d is not None)

    # parameters
    graphical_steps = [161] # specify steps
    rotation_plus = 0.47 # rotation of the frame along the lon when making plot

    # case options 
    Case_Options_2d = CASE_OPTIONS_TWOD1(local_dir_2d)
    Case_Options_2d.Interpret()
    Case_Options_2d.SummaryCaseVtuStep(os.path.join(local_dir_2d, "summary.csv"))

    # Initiate plotting class
    PlotCaseRunTwoD1(local_dir_2d, step=graphical_steps, rotation_plus=rotation_plus)

    # Processing pyvista
    if is_process_pyvista_for_plot_2d:
        # PlotCaseRunThD.ProcessPyvista()
        pass

    # Generate paraview script
    for step in graphical_steps:
        my_assert(len(graphical_steps)==1, ValueError, "Feeding the trench position only works when there is only one step")

        # Get time 
        idx = Case_Options_2d.summary_df["Vtu step"] == step
        _time = Case_Options_2d.summary_df.loc[idx, "Time"].values[0]

        # get trench center
        pvtu_step = step + int(Case_Options_2d.options['INITIAL_ADAPTIVE_REFINEMENT']) 
        pyvista_outdir = os.path.join(local_dir_2d, "pyvista_outputs", "%05d" % pvtu_step)
        # trench_center = get_trench_position_from_file(pyvista_outdir, pvtu_step, Case_Options_2d.options['GEOMETRY'])
        # slab_depth = get_slab_depth_from_file(pyvista_outdir, pvtu_step, Case_Options_2d.options['GEOMETRY'], float(Case_Options_2d.options['OUTER_RADIUS']), "sp_lower")

        # print("trench_center: ", trench_center) # debug
        # generate paraview script
        # addtional_options = {"TRENCH_CENTER": trench_center, "PLOT_TIME": _time}
        # PlotCaseRunThD.GenerateParaviewScript(ofile_list, addtional_options)

self.options['GRAPHICAL_STEPS']:  [0, 479]
PlotCaseRunTwoD1: start
self.options['GRAPHICAL_STEPS']:  [161]


KeyError: 'spcrust'